## D212 Data Mining 2 PA 3
##### Submitted By Edwin Perry
### Table of Contents
<ol>
    <li><a href="#A">Research Question</a></li>
    <li><a href="#B">Technique Justification</a></li>
    <li><a href="#C">Data Preparation and Analysis</a></li>
    <li><a href="#D">Data Summary And Implications</a></li>
    <li><a href="#E">Attachments</a></li>
    <li><a href="#F">Sources</a></li>
</ol>
<h4 id="A">Research Question</h4>
<h5>Providing Question</h5>
<p>The research question that I have decided to use for this project is "What product offerings are positively associated with the TP-Link AC1750 Smart WiFi Router?" This can be used to items commonly purchased together to offer deals, discounts, and bundles.</p>
<h5>Objectives and Goals</h5>
<p>Determining common groupings of goods provided can assist the telecommunications business in creating bundles and discount groups, encouraging customers to purchase these goods from the business.If customers can purchase these bundles from our business, we can both improve customer satisfaction with the business, encouraging return customers, and ensure customers purchase our goods, rather than purhcasing these goods from competitors. I am specifically choosing to analyze the router because installing a new router is a serious technological change, which might necessitate the use of other pieces of equipment.</p>
<h4 id="B">Technique Justification</h4>
<h5>Market Basket Description</h5>
<p>Market Basket Analysis is a way to identify the most common groupings of products purchased. This is often used to create recommendations based on these groupings, allowing a business to create bundles or deals on the basis of these common groupings.

The analysis is done by creating rules with an antecedent and consequent relationship. To put it in simple terms, we determine a value (the antecedent) and see if it has a causal relationship with the purchase of another item (the consequent).

To bring it back to the research question, I would anticipate that the result of this analysis will reveal a relationship that could be formulated as "If a customer purchases the router, then they are likely to purchase X" or "customers that purchase X are more likely to purchase a router</p>
<h5>Example Transaction</h5>
<p></p>
<h5>Assumption</h5>
